In [10]:
# Import necessary libraries
import cv2
import torch
import numpy as np
from PIL import Image
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import time

# Initialize tracking history and first player detection flag
position_history = {'x': [], 'y': [], 'frame_num': []}
first_player = None

def detect_players(frame):
    """
    Detect players wearing white shirts on the basketball court
    """
    global first_player
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define range for white clothing - more restrictive to avoid court lines
    white_lower = np.array([0, 0, 200])  # Increased value threshold
    white_upper = np.array([180, 50, 255])  # Increased saturation threshold
    
    # Create mask for white
    white_mask = cv2.inRange(hsv, white_lower, white_upper)
    
    # Remove thin lines using morphological operations
    kernel_open = np.ones((7,7), np.uint8)  # Larger kernel to remove lines
    kernel_close = np.ones((5,5), np.uint8)
    
    # Opening operation (erosion followed by dilation)
    mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel_open)
    
    # Closing operation (dilation followed by erosion)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_close)
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by area and shape
    min_area = 100 # Increased minimum area
    max_area = 300
    valid_players = []
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area < area < max_area:
            # Calculate solidity (area / convex hull area)
            hull = cv2.convexHull(contour)
            hull_area = cv2.contourArea(hull)
            solidity = float(area) / hull_area if hull_area > 0 else 0
            
            # Calculate aspect ratio
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w)/h
            
            # Filter based on shape characteristics
            if 0.3 < aspect_ratio < 2.0 and solidity > 0.5:  # Added solidity check
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    
                    # Only consider objects away from the edges
                    edge_margin = 2 #**************
                    if (edge_margin < cx < frame.shape[1]-edge_margin and 
                        edge_margin < cy < frame.shape[0]-edge_margin):
                        valid_players.append({
                            'position': (cx, cy),
                            'contour': contour,
                            'area': area
                        })
    
    # If this is our first detection
    if first_player is None and valid_players:
        # Sort by area and take the largest valid detection
        valid_players.sort(key=lambda x: x['area'], reverse=True)
        first_player = valid_players[0]
        return [first_player]
    # If we already have a player, track the closest one to last position
    elif first_player is not None and valid_players and position_history['x']:
        last_pos = (position_history['x'][-1], position_history['y'][-1])
        closest_player = min(valid_players, 
                           key=lambda p: np.sqrt((p['position'][0] - last_pos[0])**2 + 
                                               (p['position'][1] - last_pos[1])**2))
        return [closest_player]
    
    return []

def draw_tracking(frame, players, frame_num):
    """
    Draw player position and tracking information
    """
    if players:
        player = players[0]
        pos = player['position']
        contour = player['contour']
        
        # Draw contour
        cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)
        
        # Draw center point
        cv2.circle(frame, pos, 5, (255, 0, 0), -1)
        
        # Add to tracking history
        position_history['x'].append(pos[0])
        position_history['y'].append(pos[1])
        position_history['frame_num'].append(frame_num)
    
    # Draw trail
    if len(position_history['x']) > 1:
        for i in range(1, len(position_history['x'])):
            pt1 = (position_history['x'][i-1], position_history['y'][i-1])
            pt2 = (position_history['x'][i], position_history['y'][i])
            cv2.line(frame, pt1, pt2, (0, 0, 255), 2)
    
    return frame

def process_video(video_path):
    """
    Process basketball court video with player tracking
    """
    global first_player
    
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0:
        fps = 30
    
    # Create output video writer
    output_path = 'football_tracking.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    frame_num = 0
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            display_frame = frame.copy()
            players = detect_players(frame)
            frame_with_tracking = draw_tracking(display_frame, players, frame_num)
            out.write(frame_with_tracking)
            frame_num += 1
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print("Processing complete!")
        print("Output video saved as 'single_player_tracking.mp4'")

# Reset global variables
first_player = None
position_history = {'x': [], 'y': [], 'frame_num': []}

# Run the processing
video_path = 'football.mp4'  # Replace with your video path
process_video(video_path)

Processing complete!
Output video saved as 'single_player_tracking.mp4'


In [28]:
import cv2
import numpy as np
from typing import List, Dict

class HelmetTracker:
    def __init__(self):
        self.players: Dict[int, Dict] = {}
        self.max_distance = 50
        self.initialized = False
        self.num_players_to_track = 2
        
    def detect_helmets(self, frame: np.ndarray) -> List[Dict]:
        """
        Detect white football helmets using circular shape detection
        """
        # Convert to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Refined range for white helmets
        white_lower = np.array([0, 0, 180])
        white_upper = np.array([180, 30, 255])
        white_mask = cv2.inRange(hsv, white_lower, white_upper)
        
        # Morphological operations to clean up the mask
        kernel = np.ones((3,3), np.uint8)
        mask = cv2.morphologyEx(white_mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Parameters for helmet detection
        min_area = 20  # Minimum area for a helmet
        max_area = 100  # Maximum area for a helmet
        min_circularity = 0.75 # Minimum circularity (1.0 is perfect circle)
        detected_helmets = []
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if min_area < area < max_area:
                # Calculate circularity
                perimeter = cv2.arcLength(contour, True)
                circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
                
                if circularity > min_circularity:
                    M = cv2.moments(contour)
                    if M["m00"] != 0:
                        cx = int(M["m10"] / M["m00"])
                        cy = int(M["m01"] / M["m00"])
                        
                        detected_helmets.append({
                            'position': (cx, cy),
                            'contour': contour,
                            'area': area,
                            'circularity': circularity,
                            'matched': False
                        })
        
        return detected_helmets
    
    def select_initial_helmets(self, detected_helmets: List[Dict]) -> List[Dict]:
        """
        Select initial helmets to track based on circularity and spacing
        """
        if len(detected_helmets) < self.num_players_to_track:
            return detected_helmets
            
        # Sort by circularity (most circular first)
        sorted_helmets = sorted(detected_helmets, key=lambda h: (-h['circularity'], h['position'][0]))
        
        selected_helmets = []
        min_distance = 300  # Minimum pixels between selected helmets
        
        for helmet in sorted_helmets:
            if not any(self._distance(helmet['position'], h['position']) < min_distance 
                      for h in selected_helmets):
                selected_helmets.append(helmet)
                if len(selected_helmets) == self.num_players_to_track:
                    break
        
        return selected_helmets[:self.num_players_to_track]
    
    def _distance(self, pos1, pos2):
        return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)
    
    def update_tracks(self, detected_helmets: List[Dict]):
        """
        Update helmet tracks using detected positions
        """
        if not self.initialized:
            selected_helmets = self.select_initial_helmets(detected_helmets)
            for i, helmet in enumerate(selected_helmets):
                self.players[i] = {
                    'positions': [helmet['position']],
                    'last_seen': 0
                }
            self.initialized = True
            return
        
        matched_tracks = set()
        
        for helmet in detected_helmets:
            helmet['matched'] = False
        
        for track_id, track in self.players.items():
            if track_id in matched_tracks:
                continue
                
            min_dist = float('inf')
            best_match = None
            
            for i, helmet in enumerate(detected_helmets):
                if helmet['matched']:
                    continue
                    
                dist = self._distance(helmet['position'], track['positions'][-1])
                
                if dist < min_dist and dist < self.max_distance:
                    min_dist = dist
                    best_match = i
            
            if best_match is not None:
                helmet = detected_helmets[best_match]
                self.players[track_id]['positions'].append(helmet['position'])
                self.players[track_id]['last_seen'] = 0
                helmet['matched'] = True
                matched_tracks.add(track_id)
            else:
                self.players[track_id]['positions'].append(self.players[track_id]['positions'][-1])
                self.players[track_id]['last_seen'] += 1

    def draw_tracks(self, frame: np.ndarray) -> np.ndarray:
        """
        Draw helmet positions and trails
        """
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Red, Green, Blue for different players
        
        for track_id, track in self.players.items():
            positions = track['positions']
            color = colors[track_id % len(colors)]
            
            # Draw trail
            if len(positions) > 1:
                for i in range(1, len(positions)):
                    pt1 = positions[i-1]
                    pt2 = positions[i]
                    cv2.line(frame, pt1, pt2, color, 2)
            
            # Draw current position
            if positions:
                cv2.circle(frame, positions[-1], 5, color, -1)
                cv2.putText(frame, f"Player {track_id + 1}", 
                          (positions[-1][0] + 5, positions[-1][1] + 5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        return frame

def process_video(video_path: str, output_path: str):
    """
    Process football video with white helmet tracking
    """
    cap = cv2.VideoCapture(video_path)
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0:
        fps = 30
        
    tracker = HelmetTracker()
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            display_frame = frame.copy()
            detected_helmets = tracker.detect_helmets(frame)
            tracker.update_tracks(detected_helmets)
            frame_with_tracking = tracker.draw_tracks(display_frame)
            
            out.write(frame_with_tracking)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        print(f"Processing complete! Output saved as '{output_path}'")

if __name__ == "__main__":
    video_path = 'football.mp4'
    output_path = 'white_helmet_tracking_output.mp4'
    process_video(video_path, output_path)

Processing complete! Output saved as 'white_helmet_tracking_output.mp4'
